In [1]:
%load_ext autoreload
%autoreload 2
from indices import MainDataset
from neighbors import Neighborhoods
import functions
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from catboost import Pool
from metric import metrics_stat

In [2]:
train_dataset = MainDataset("dataset/train.csv")
test_dataset = MainDataset("dataset/test.csv", need_index = False)

/Users/ivan.bragin/projects/raif/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
neighborhoods = Neighborhoods(train_dataset.index)
neighborhoods1 = Neighborhoods(train_dataset.index1)

In [4]:
# for i, o in enumerate(train_dataset.all_objects[:10000]):
#     if i % 1000 == 0:
#         print(i)
#     neighbor = neighborhoods.get_haversine_closest(o, 7)
#     for f_name, f in functions.items():
#         prices = [f(o, x[0], x[1]) for x in neighbor]
#         o.neighbors_scores[f_name] = np.mean(prices)

In [5]:
for prefix, neighborhoods_item in [("", neighborhoods), ("P1_", neighborhoods1)]:
    for i, o in enumerate(train_dataset.all_objects):
#         o.neighbors_scores = {}
        if o.row["price_type"] != 1:
            continue
        if i % 1000 == 0:
            print(i)
        neighbor = neighborhoods_item.get_haversine_closest(o, 7)
        for f_name, f in functions.functions.items():
            prices = [f(o, x[0], x[1]) for x in neighbor]
            prices = [x for x in prices if x > 0]
            if len(prices) > 0:
                price_median = np.median(prices)
                price_max = np.max(prices)
                price_min = np.min(prices)

            else:
                price_median = 0
                price_max = 0
                price_min = 0
            o.neighbors_scores[prefix + f_name+"_median"] = price_median
            o.neighbors_scores[prefix + f_name+"_max"] = price_max
            o.neighbors_scores[prefix + f_name+"_min"] = price_min
    

253000
254000
255000
256000
253000
254000
255000
256000


In [6]:
for prefix, neighborhoods_item in [("", neighborhoods), ("P1_", neighborhoods1)]:
    for i, o in enumerate(test_dataset.all_objects):
        if o.row["price_type"] != 1:
            continue
        if i % 1000 == 0:
            print(i)
        neighbor = neighborhoods_item.get_haversine_closest(o, 7)
        for f_name, f in functions.functions.items():
            prices = [f(o, x[0], x[1]) for x in neighbor]
            prices = [x for x in prices if x > 0]
            if len(prices) > 0:
                price_median = np.median(prices)
                price_max = np.max(prices)
                price_min = np.min(prices)

            else:
                price_median = 0
                price_max = 0
                price_min = 0
            o.neighbors_scores[prefix + f_name+"_median"] = price_median
            o.neighbors_scores[prefix + f_name+"_max"] = price_max
            o.neighbors_scores[prefix + f_name+"_min"] = price_min

0
1000
2000
0
1000
2000


In [46]:
raw_dense_features = [
#     'osm_historic_points_in_0.005',
#     'osm_culture_points_in_0.0075',
        'osm_amenity_points_in_0.001',
       'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075',
       'osm_amenity_points_in_0.01', 'osm_building_points_in_0.001',
       'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
       'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
       'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075',
       'osm_catering_points_in_0.01', 'osm_city_closest_dist',
        'osm_city_nearest_population',
       'osm_crossing_closest_dist', 'osm_crossing_points_in_0.001',
       'osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075',
       'osm_crossing_points_in_0.01', 'osm_culture_points_in_0.001',
       'osm_culture_points_in_0.005', 
       'osm_culture_points_in_0.01', 'osm_finance_points_in_0.001',
       'osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075',
       'osm_finance_points_in_0.01', 'osm_healthcare_points_in_0.005',
       'osm_healthcare_points_in_0.0075', 'osm_healthcare_points_in_0.01',
        'osm_historic_points_in_0.0075',
       'osm_historic_points_in_0.01', 'osm_hotels_points_in_0.005',
       'osm_hotels_points_in_0.0075', 'osm_hotels_points_in_0.01',
       'osm_leisure_points_in_0.005', 'osm_leisure_points_in_0.0075',
       'osm_leisure_points_in_0.01', 'osm_offices_points_in_0.001',
       'osm_offices_points_in_0.005', 'osm_offices_points_in_0.0075',
       'osm_offices_points_in_0.01', 'osm_shops_points_in_0.001',
       'osm_shops_points_in_0.005', 'osm_shops_points_in_0.0075',
       'osm_shops_points_in_0.01', 'osm_subway_closest_dist',
       'osm_train_stop_closest_dist', 'osm_train_stop_points_in_0.005',
       'osm_train_stop_points_in_0.0075', 'osm_train_stop_points_in_0.01',
       'osm_transport_stop_closest_dist', 'osm_transport_stop_points_in_0.005',
       'osm_transport_stop_points_in_0.0075',
       'osm_transport_stop_points_in_0.01',
                      'reform_count_of_houses_1000', 'reform_count_of_houses_500',
       'reform_house_population_1000', 'reform_house_population_500',
       'reform_mean_floor_count_1000', 'reform_mean_floor_count_500',
       'reform_mean_year_building_1000', 'reform_mean_year_building_500',
       'total_square', "lat", "lng"
       ]

raw_cat_features = ['city', 'osm_city_nearest_name', 'region', 'realty_type', 'street', 'floor']

bad_f = set(["F_square_False_min", "F_square_False_median"])
def getX(o):
    x = []
    for f_name, score in o.neighbors_scores.items():
        if f_name not in bad_f and "P1_" not in f_name:
            x.append(score)
    for n in raw_dense_features:
        x.append(o.row[n])
    for n in raw_cat_features:
        x.append(o.row[n])
    return pd.Series(x)


In [47]:
columns = []
for p in ["",
#           "P1_"
         ]:
    for t in ["_median", "_min", "_max"]:
        columns.extend([p + f_name + t for f_name, f in functions.functions.items() ])
columns = [c for c in columns if c not in bad_f]
    # columns = []
columns.extend(raw_dense_features)
columns.extend(raw_cat_features)

In [48]:
# X0 = [getX(o) for o in train_dataset.all_objects[:1] if o.row["price_type"] == 0]
# X0 = pd.concat(X0, axis=1).T
# X0.columns=columns
# Y0 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 0]
# X0["floor"] = X0["floor"].astype("str")

In [49]:
X1 = [getX(o) for o in train_dataset.all_objects if o.row["price_type"] == 1]
X1 = pd.concat(X1, axis=1).T
X1.columns=columns
Y1 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 1]
X1 = X1.fillna(-999)


In [50]:
Xtest1 = [getX(o) for o in test_dataset.all_objects]
Xtest1 = pd.concat(Xtest1, axis=1).T
Xtest1.columns=columns
Xtest1 = Xtest1.fillna(-999)

In [51]:
# X0 = np.stack([o.getX() for o in train_dataset.all_objects if o.row["price_type"] == 0])
# X1 = np.stack([o.getX() for o in train_dataset.all_objects if o.row["price_type"] == 1])
# Y1 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 1]

In [52]:
# np.save("x0", X0)
# np.save("y0", Y0)
# np.save("x1", X1)
# np.save("y1", Y1)

In [53]:
# X0 = X0.fillna(-999)
X_train, X_val, Y_train, Y_val = train_test_split(X1, Y1, test_size=0.1, random_state=42)

In [135]:
train_data = Pool(data=X_train,
                  cat_features=raw_cat_features,
                  label=np.log(np.array(Y_train)))

val_data = Pool(data=X_val,
                cat_features=raw_cat_features,
                  label=np.log(np.array(Y_val)))

In [136]:
class RmseObjective(object):
    def calc_ders_range(self, approxes, targets, weights):
        assert len(approxes) == len(targets)
        
        result = []
        for index in range(len(targets)):
            y = targets[index]
            p = approxes[index]
            if y < p:
                der1 = 1.1*(y-p + 0.15)
                der2 = -1.1
            else:
                der1 = y-p-0.15
                der2 = -1
       

            result.append((der1, der2))
        return result

In [ ]:
cat_model = CatBoostRegressor(
    loss_function=RmseObjective(),
    learning_rate=0.01,
    iterations=8000,
    eval_metric="RMSE",
)
cat_model.fit(train_data,eval_set=val_data, use_best_model=False)



Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 10.8161056	test: 10.7728196	best: 10.7728196 (0)	total: 115ms	remaining: 15m 23s
1:	learn: 10.7111689	test: 10.6676204	best: 10.6676204 (1)	total: 224ms	remaining: 14m 57s
2:	learn: 10.6079085	test: 10.5645180	best: 10.5645180 (2)	total: 335ms	remaining: 14m 52s
3:	learn: 10.5057514	test: 10.4619914	best: 10.4619914 (3)	total: 442ms	remaining: 14m 43s
4:	learn: 10.4038066	test: 10.3599243	best: 10.3599243 (4)	total: 543ms	remaining: 14m 27s
5:	learn: 10.3031893	test: 10.2596310	best: 10.2596310 (5)	total: 649ms	remaining: 14m 24s
6:	learn: 10.2029060	test: 10.1595915	best: 10.1595915 (6)	total: 774ms	remaining: 14m 43s
7:	learn: 10.1049690	test: 10.0622079	best: 10.0622079 (7)	total: 945ms	remaining: 15m 43s
8:	learn: 10.0067036	test: 9.9643552	best: 9.9643552 (8)	total: 1.11s	remaining: 16m 27s
9:	learn: 9.9098530	test: 9.8678220	best: 9.8678220 (9)	total: 1.26s	remaining: 16m 46s
10:	learn: 9.8138554	test: 9.7716831	best: 9.7716831 (10)	total: 1.44s	remaining: 17m 26s
11:	l

94:	learn: 4.3767555	test: 4.3394615	best: 4.3394615 (94)	total: 10.5s	remaining: 14m 34s
95:	learn: 4.3352984	test: 4.2979802	best: 4.2979802 (95)	total: 10.6s	remaining: 14m 33s
96:	learn: 4.2948790	test: 4.2579696	best: 4.2579696 (96)	total: 10.7s	remaining: 14m 35s
97:	learn: 4.2545961	test: 4.2176481	best: 4.2176481 (97)	total: 10.9s	remaining: 14m 35s
98:	learn: 4.2150323	test: 4.1780824	best: 4.1780824 (98)	total: 11s	remaining: 14m 35s
99:	learn: 4.1755429	test: 4.1384439	best: 4.1384439 (99)	total: 11.1s	remaining: 14m 35s
100:	learn: 4.1365967	test: 4.0999110	best: 4.0999110 (100)	total: 11.2s	remaining: 14m 34s
101:	learn: 4.0982225	test: 4.0616050	best: 4.0616050 (101)	total: 11.3s	remaining: 14m 34s
102:	learn: 4.0596555	test: 4.0231051	best: 4.0231051 (102)	total: 11.4s	remaining: 14m 33s
103:	learn: 4.0219808	test: 3.9854304	best: 3.9854304 (103)	total: 11.5s	remaining: 14m 33s
104:	learn: 3.9843021	test: 3.9482422	best: 3.9482422 (104)	total: 11.6s	remaining: 14m 32s
10

185:	learn: 1.9120580	test: 1.8784308	best: 1.8784308 (185)	total: 20.6s	remaining: 14m 27s
186:	learn: 1.8954898	test: 1.8619393	best: 1.8619393 (186)	total: 20.8s	remaining: 14m 27s
187:	learn: 1.8791679	test: 1.8457645	best: 1.8457645 (187)	total: 20.9s	remaining: 14m 27s
188:	learn: 1.8629292	test: 1.8295644	best: 1.8295644 (188)	total: 21s	remaining: 14m 27s
189:	learn: 1.8469739	test: 1.8136415	best: 1.8136415 (189)	total: 21.1s	remaining: 14m 26s
190:	learn: 1.8311741	test: 1.7978589	best: 1.7978589 (190)	total: 21.2s	remaining: 14m 26s
191:	learn: 1.8155110	test: 1.7823024	best: 1.7823024 (191)	total: 21.3s	remaining: 14m 26s
192:	learn: 1.8000831	test: 1.7668673	best: 1.7668673 (192)	total: 21.4s	remaining: 14m 26s
193:	learn: 1.7847746	test: 1.7515633	best: 1.7515633 (193)	total: 21.5s	remaining: 14m 26s
194:	learn: 1.7694682	test: 1.7363135	best: 1.7363135 (194)	total: 21.6s	remaining: 14m 26s
195:	learn: 1.7543497	test: 1.7213691	best: 1.7213691 (195)	total: 21.7s	remaining

276:	learn: 0.9415219	test: 0.9111153	best: 0.9111153 (276)	total: 30s	remaining: 13m 56s
277:	learn: 0.9353284	test: 0.9048596	best: 0.9048596 (277)	total: 30.1s	remaining: 13m 56s
278:	learn: 0.9291698	test: 0.8987851	best: 0.8987851 (278)	total: 30.3s	remaining: 13m 57s
279:	learn: 0.9230490	test: 0.8926184	best: 0.8926184 (279)	total: 30.4s	remaining: 13m 57s
280:	learn: 0.9170160	test: 0.8865400	best: 0.8865400 (280)	total: 30.5s	remaining: 13m 57s
281:	learn: 0.9110947	test: 0.8806644	best: 0.8806644 (281)	total: 30.6s	remaining: 13m 58s
282:	learn: 0.9051762	test: 0.8748660	best: 0.8748660 (282)	total: 30.7s	remaining: 13m 58s
283:	learn: 0.8992527	test: 0.8690268	best: 0.8690268 (283)	total: 30.9s	remaining: 13m 58s
284:	learn: 0.8933874	test: 0.8631353	best: 0.8631353 (284)	total: 31s	remaining: 13m 58s
285:	learn: 0.8876823	test: 0.8575623	best: 0.8575623 (285)	total: 31.1s	remaining: 13m 57s
286:	learn: 0.8820447	test: 0.8519742	best: 0.8519742 (286)	total: 31.2s	remaining: 

...

1525:	learn: 0.3002127	test: 0.3278490	best: 0.3278490 (1525)	total: 2m 34s	remaining: 10m 57s
1526:	learn: 0.3001888	test: 0.3278344	best: 0.3278344 (1526)	total: 2m 35s	remaining: 10m 57s
1527:	learn: 0.3001302	test: 0.3278125	best: 0.3278125 (1527)	total: 2m 35s	remaining: 10m 57s
1528:	learn: 0.3001116	test: 0.3277977	best: 0.3277977 (1528)	total: 2m 35s	remaining: 10m 57s
1529:	learn: 0.3000696	test: 0.3277743	best: 0.3277743 (1529)	total: 2m 35s	remaining: 10m 57s
1530:	learn: 0.3000459	test: 0.3277596	best: 0.3277596 (1530)	total: 2m 35s	remaining: 10m 57s
1531:	learn: 0.3000118	test: 0.3277311	best: 0.3277311 (1531)	total: 2m 35s	remaining: 10m 56s
1532:	learn: 0.2999915	test: 0.3277250	best: 0.3277250 (1532)	total: 2m 35s	remaining: 10m 56s
1533:	learn: 0.2999623	test: 0.3277124	best: 0.3277124 (1533)	total: 2m 35s	remaining: 10m 56s
1534:	learn: 0.2999561	test: 0.3277012	best: 0.3277012 (1534)	total: 2m 35s	remaining: 10m 56s
1535:	learn: 0.2999297	test: 0.3276866	best: 0.327

1612:	learn: 0.2973343	test: 0.3267732	best: 0.3267732 (1612)	total: 2m 43s	remaining: 10m 46s
1613:	learn: 0.2973007	test: 0.3267523	best: 0.3267523 (1613)	total: 2m 43s	remaining: 10m 46s
1614:	learn: 0.2972519	test: 0.3267414	best: 0.3267414 (1614)	total: 2m 43s	remaining: 10m 46s
1615:	learn: 0.2972156	test: 0.3267406	best: 0.3267406 (1615)	total: 2m 43s	remaining: 10m 45s
1616:	learn: 0.2971574	test: 0.3267342	best: 0.3267342 (1616)	total: 2m 43s	remaining: 10m 45s
1617:	learn: 0.2971519	test: 0.3267205	best: 0.3267205 (1617)	total: 2m 43s	remaining: 10m 45s
1618:	learn: 0.2971214	test: 0.3267078	best: 0.3267078 (1618)	total: 2m 43s	remaining: 10m 45s
1619:	learn: 0.2970766	test: 0.3267193	best: 0.3267078 (1618)	total: 2m 43s	remaining: 10m 45s
1620:	learn: 0.2970405	test: 0.3267114	best: 0.3267078 (1618)	total: 2m 43s	remaining: 10m 45s
1621:	learn: 0.2969838	test: 0.3267135	best: 0.3267078 (1618)	total: 2m 44s	remaining: 10m 45s
1622:	learn: 0.2969486	test: 0.3267072	best: 0.326

1699:	learn: 0.2943710	test: 0.3258038	best: 0.3258038 (1699)	total: 2m 51s	remaining: 10m 35s
1700:	learn: 0.2943660	test: 0.3257942	best: 0.3257942 (1700)	total: 2m 51s	remaining: 10m 35s
1701:	learn: 0.2943373	test: 0.3257923	best: 0.3257923 (1701)	total: 2m 51s	remaining: 10m 34s
1702:	learn: 0.2943045	test: 0.3257977	best: 0.3257923 (1701)	total: 2m 51s	remaining: 10m 34s
1703:	learn: 0.2942630	test: 0.3257726	best: 0.3257726 (1703)	total: 2m 51s	remaining: 10m 34s
1704:	learn: 0.2942066	test: 0.3257653	best: 0.3257653 (1704)	total: 2m 51s	remaining: 10m 34s
1705:	learn: 0.2941780	test: 0.3257587	best: 0.3257587 (1705)	total: 2m 51s	remaining: 10m 34s
1706:	learn: 0.2941482	test: 0.3257394	best: 0.3257394 (1706)	total: 2m 52s	remaining: 10m 34s
1707:	learn: 0.2941132	test: 0.3257038	best: 0.3257038 (1707)	total: 2m 52s	remaining: 10m 34s
1708:	learn: 0.2941083	test: 0.3256957	best: 0.3256957 (1708)	total: 2m 52s	remaining: 10m 34s
1709:	learn: 0.2940794	test: 0.3256972	best: 0.325

1786:	learn: 0.2914503	test: 0.3248009	best: 0.3248009 (1786)	total: 2m 59s	remaining: 10m 24s
1787:	learn: 0.2914059	test: 0.3248133	best: 0.3248009 (1786)	total: 2m 59s	remaining: 10m 24s
1788:	learn: 0.2913617	test: 0.3247819	best: 0.3247819 (1788)	total: 2m 59s	remaining: 10m 24s
1789:	learn: 0.2913292	test: 0.3247805	best: 0.3247805 (1789)	total: 2m 59s	remaining: 10m 24s
1790:	learn: 0.2912905	test: 0.3247703	best: 0.3247703 (1790)	total: 2m 59s	remaining: 10m 23s
1791:	learn: 0.2912653	test: 0.3247696	best: 0.3247696 (1791)	total: 3m	remaining: 10m 23s
1792:	learn: 0.2912197	test: 0.3247534	best: 0.3247534 (1792)	total: 3m	remaining: 10m 23s
1793:	learn: 0.2911878	test: 0.3247607	best: 0.3247534 (1792)	total: 3m	remaining: 10m 23s
1794:	learn: 0.2911440	test: 0.3247202	best: 0.3247202 (1794)	total: 3m	remaining: 10m 23s
1795:	learn: 0.2911209	test: 0.3247097	best: 0.3247097 (1795)	total: 3m	remaining: 10m 23s
1796:	learn: 0.2911160	test: 0.3246975	best: 0.3246975 (1796)	total: 3

1876:	learn: 0.2883610	test: 0.3236088	best: 0.3236088 (1876)	total: 3m 8s	remaining: 10m 13s
1877:	learn: 0.2883179	test: 0.3235921	best: 0.3235921 (1877)	total: 3m 8s	remaining: 10m 13s
1878:	learn: 0.2882818	test: 0.3235828	best: 0.3235828 (1878)	total: 3m 8s	remaining: 10m 13s
1879:	learn: 0.2882534	test: 0.3235739	best: 0.3235739 (1879)	total: 3m 8s	remaining: 10m 13s
1880:	learn: 0.2882208	test: 0.3235681	best: 0.3235681 (1880)	total: 3m 8s	remaining: 10m 12s
1881:	learn: 0.2881680	test: 0.3235346	best: 0.3235346 (1881)	total: 3m 8s	remaining: 10m 12s
1882:	learn: 0.2881257	test: 0.3235184	best: 0.3235184 (1882)	total: 3m 8s	remaining: 10m 12s
1883:	learn: 0.2880992	test: 0.3235196	best: 0.3235184 (1882)	total: 3m 8s	remaining: 10m 12s
1884:	learn: 0.2880638	test: 0.3234923	best: 0.3234923 (1884)	total: 3m 8s	remaining: 10m 12s
1885:	learn: 0.2880207	test: 0.3234781	best: 0.3234781 (1885)	total: 3m 8s	remaining: 10m 12s
1886:	learn: 0.2879923	test: 0.3234793	best: 0.3234781 (1885

In [117]:
pd.set_option("display.max_rows", None) 
cat_model.get_feature_importance(data=val_data,prettified=True)


,Feature Id,Importances
0,total_square,6.161665
1,region,5.134578
2,city,4.525712
3,osm_city_nearest_name,4.061817
4,floor,2.956830
5,osm_crossing_closest_dist,2.563412
6,lng,2.283491
7,osm_transport_stop_closest_dist,2.111620
8,realty_type,1.922373
9,street,1.888108


In [118]:
price0_preds = cat_model.predict(X1)
metrics_stat(np.array(price0_preds) , np.array(Y1))

{'mape': 6166.761312318053,
 'mdape': 4700.85705258392,
 'rmse': 113026.82775046682,
 'r2': -41633125248.57285,
 'raif_metric': 9.899999999999999}

In [128]:
price0_preds = cat_model.predict(X_val, ntree_end=12000)
metrics_stat(np.array(Y_val), np.exp(np.array(price0_preds)))

{'mape': 0.21794975875705683,
 'mdape': 0.16521368203426817,
 'rmse': 48267.042666062385,
 'r2': 0.6737439998109604,
 'raif_metric': 1.2618318341695673}

In [59]:
Xtest1 = Xtest1.fillna(-999)
test_data = Pool(data=Xtest1,
                  cat_features=raw_cat_features)
price0_preds =  np.exp(cat_model.predict(test_data))/1.15

In [61]:
pd.DataFrame({"id": [o.row['id'] for o in test_dataset.all_objects],
             "per_square_meter_price": price0_preds
             }).to_csv("log2.csv", index=False)

In [ ]:
train_data = Pool(data=X_train,
                  cat_features=raw_cat_features,
                  label=np.log(np.array(Y_train)*0.85))

val_data = Pool(data=X_val,
                cat_features=raw_cat_features,
                  label=np.log(np.array(Y_val)*0.85))

In [130]:
import lightgbm as lgb

Looking in indexes: https://pypi.org/simple, https://nexus.odkl.ru/repository/pypi-dev-hosted/simple
  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7fa83d117048>, 'Connection to nexus.odkl.ru timed out. (connect timeout=15)')': /repository/pypi-dev-hosted/simple/lightgbm/
  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7fa83d1170f0>, 'Connection to nexus.odkl.ru timed out. (connect timeout=15)')': /repository/pypi-dev-hosted/simple/lightgbm/
  Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7fa83d1171d0>, 'Connection to nexus.odkl.ru timed out. (co

In [129]:
lgb_train = lgb.Dataset(X_train, np.log(np.array(Y_train)*0.85), categorical_feature=raw_cat_features)
lgb_eval = lgb.Dataset(X_val, np.log(np.array(Y_val)*0.85), reference=lgb_train, categorical_feature=raw_cat_features)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
#     'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)



NameError: name 'lgb' is not defined